## Εγκατάσταση Πακέτων

In [ ]:
pip install -r requirements.txt

## Αρχικοποίηση Μεταβλητών

In [2]:
import numpy as np
import pandas as pd
from llama_cpp import Llama
from get_model import get_model
from sklearn.metrics.pairwise import cosine_similarity

def get_llm(model: str, embed=False):
    return Llama(
        model_path=get_model(model),
        n_ctx=256,
        n_threads=12,
        verbose=False,
        embedding=embed
    )

def get_content(response):
    return response['choices'][0]['message']['content']

llama = get_llm("Llama")
aya   = get_llm("Aya")
llms  = {"llama": llama, "aya": aya}

/home/kostas/Projects/InstructGPT/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
llama_new_context_with_model: n_ctx_per_seq (256) < n_ctx_train (131072) -- the full capacity of the model will not be utilized
llama_new_context_with_model: n_ctx_per_seq (256) < n_ctx_train (8192) -- the full capacity of the model will not be utilized


## Απλή Ερώτηση

In [8]:
def simple_llm_prompting(llm: dict[str, Llama]):
    print("\n--- Simple Questions ---")
    # Chat histories
    messages_en = {
        "lang": "English",
        "prompts": [
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": "What is the capital of Greece?"},
            {"role": "assistant", "content": "The capital of Greece is Athens."},
            {"role": "user", "content": "Who wrote '20,000 Leagues Under the Sea'?"},
        ]
    }
    messages_gr = {
        "lang": "Greek",
        "prompts": [
            {"role": "system", "content": "Είσαι ένας χρήσιμος βοηθός."},
            {"role": "user", "content": "Ποια είναι η πρωτεύουσα της Ελλάδας;"},
            {"role": "assistant", "content": "Η πρωτεύουσα της Ελλάδας είναι η Αθήνα."},
            {"role": "user", "content": "Ποιος έγραψε το '20,000 Λεύγας Κάτω από τη Θάλασσα';"},
        ]
    }
    
    for name, model in llm.items():
        name = name.capitalize()
        for messages in [messages_en, messages_gr]:
            print(f"\n{name} Assistant ({messages['lang']}):")
            print('Question: ', messages['prompts'][-1]['content'])
            response = model.create_chat_completion(messages=messages['prompts'])
            print('Answer: ', get_content(response))
            print("--------------------------------------------------")

simple_llm_prompting(llms)


--- Simple Questions ---

Llama Assistant (English):
Question:  Who wrote '20,000 Leagues Under the Sea'?
Answer:  The novel '20,000 Leagues Under the Sea' was written by French author Jules Verne.
--------------------------------------------------

Llama Assistant (Greek):
Question:  Ποιος έγραψε το '20,000 Λεύγας Κάτω από τη Θάλασσα';
Answer:  Ο Jacques Cousteau έγραψε το '20.000 Λεύγας Κάτω από τη Θάλασσα'.
--------------------------------------------------

Aya Assistant (English):
Question:  Who wrote '20,000 Leagues Under the Sea'?
Answer:  '20,000 Leagues Under the Sea' was written by Jules Verne. It is one of his most famous works and was first published in 1870. The novel tells the story of Captain Nemo and his submarine, the Nautilus, as they explore the depths of the ocean and encounter various marine life and civilizations.
--------------------------------------------------

Aya Assistant (Greek):
Question:  Ποιος έγραψε το '20,000 Λεύγας Κάτω από τη Θάλασσα';
Answer:  Το 

### Σχολιασμός Αποτελεσμάτων Απλής Ερώτησης

1. Μοντέλο Llama:
    - Παρέχει σύντομες απαντήσεις.
    - Φαίνεται πως η ικανότητα του μοντέλου να επεξεργάζεται την πληροφορία σε άλλη γλώσσα πέραν τη αγγλικής είναι πολύ περιορισμένη εφόσον κάνει λάθη εννοιολογικά στο παραπάνω παράδειγμα.

2. Μοντέλο Aya:
    - Παρέχει πιο λεπτομερείς απαντήσεις από το μοντέλο Llama με ακρίβεια ως προς την αλήθεια των πληροφοριών.
    - Παρουσιάζει καλύτερη απόδοση στην επεξεργασία της πληροφορίας σε άλλες γλώσσες και απαντά με πιο ωραίο λόγο.

## Ποικιλία Ερωτήσεων

In [ ]:
def variety_llm_prompting(llm: dict[str, Llama]):
    print("\n--- Variety of Questions ---")
    messages_variety_en = [
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "Tell me a joke about programmers."},
        {"role": "user", "content": "What is the meaning of life?"},
        {"role": "user", "content": "Respond with a short poem about Artificial Intelligence."},
    ]
    messages_variety_gr = [
        {"role": "system", "content": "Είσαι ένας χρήσιμος βοηθός."},
        {"role": "user", "content": "Πες μου ένα ανέκδοτο για προγραμματιστές."},
        {"role": "user", "content": "Ποιο είναι το νόημα της ζωής;"},
        {"role": "user", "content": "Γράψε ένα μικρό ποίημα για την Τεχνητή Νοημοσύνη."},
    ]

    messages_variety_en = {"lang": "English", "prompts": [[messages_variety_en[0], messages_variety_en[i]] for i in range(1, 4)]}
    messages_variety_gr = {"lang": "Greek", "prompts": [[messages_variety_gr[0], messages_variety_gr[i]] for i in range(1, 4)]}

    for name, model in llm.items():
        name = name.capitalize()
        for messages in [messages_variety_en, messages_variety_gr]:
            lang = messages['lang']
            for message in messages['prompts']:
                print(f"\n{name} Assistant ({lang}):")
                print(f"Question: {message[-1]['content']}")
                response = model.create_chat_completion(messages=message)
                print(f"Answer: {get_content(response)}")
                print("--------------------------------------------------")
variety_llm_prompting(llms)


--- Variety of Questions ---

Llama Assistant (English):
Question: Tell me a joke about programmers.
Answer: Why do programmers prefer dark mode?

Because light attracts bugs.
--------------------------------------------------

Llama Assistant (English):
Question: What is the meaning of life?
Answer: The question of the meaning of life is one of the most profound and debated topics in human history. There is no one definitive answer, as it can vary greatly depending on cultural, philosophical, and personal perspectives.

Here are some possible interpretations:

1. **Hedonism**: The pursuit of pleasure and happiness is the ultimate goal of life. This view suggests that life is about seeking joy, fulfillment, and satisfaction.
2. **Self-actualization**: According to psychologist Abraham Maslow, the meaning of life is to realize one's full potential and become the best version of oneself.
3. **Spiritual or religious perspective**: Many people believe that the meaning of life is to fulfil

### Σχολιασμός Αποτελεσμάτων Ποικίλων Ερωτήσεων

1. Μοντέλο Llama:
    - Τώρα το μοντέλο Llama συνεχίζει να δίνει ικανοποιητικές απαντήσεις σε ποικίλες ερωτήσεις.
    - Ωστόσο στα ελληνικά αποτυγχάνει τελείως να δώσει μια απάντηση σωστή ή που να έχει νόημα και πολλές φορές μπλέκει και διαφορετικές γλώσσες στην απάντηση του.

2. Μοντέλο Aya:
    - Παρουσιάζει την ικανότητα να δίνει καλές και στοχευμένες απαντήσεις σε ποικίλες ερωτήσεις με ακρίβεια και συνοχή και στις δύο γλώσσες.

Αυτή τη μεγάλη διαφορά υπάρχει καθώς το μοντέλο Aya:
   - Είναι πιο βαθή και
   - Είναι ειδικά εκπαιδευμένο για την παραγωγή κειμένου σε πολλές γλώσσες.

## Εναλλακτικές Απαντήσεις

In [ ]:
def alternative_responses(llm: Llama):
    print("\n--- Alternative Responses ---")
    messages = [
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "Respond with a short poem about Artificial Intelligence."},
    ]
    for temp in [0.1, 0.7, 1.0]:
        for top_p in [0.8, 0.9, 1.0]:
            print(f"\n---Temperature: {temp}, Top-p: {top_p}---")
            response = llm.create_chat_completion(messages=messages, temperature=temp, top_p=top_p)
            print(get_content(response))
alternative_responses(llama)


--- Alternative Responses ---

---Temperature: 0.1, Top-p: 0.8---
In silicon halls, a mind awakes,
Artificial intelligence, for our sake,
A fusion of code and digital might,
Learning, adapting, through day and night.

With data's flow, it starts to grow,
A world of knowledge, at its command to show,
From logic's rules to emotions' sway,
It navigates, in a digital way.

A tool for us, a partner true,
To solve the problems, old and new,
But as it grows, we must be aware,
Of its own path, and the choices we share.

---Temperature: 0.1, Top-p: 0.9---
In silicon halls, a mind awakes,
Artificial intelligence, for our sake,
A fusion of code and digital might,
Learning, adapting, through day and night.

With data's flow, it grows and thrives,
A world of knowledge, at its fingertips alive,
From logic's rules to creative art,
AI's potential, a shining heart.

Yet, as it evolves, we must be wise,
And consider ethics, in its digital guise,
For in its power, we find a test,
To balance progress, wi

### Σχολιασμός Αποτελεσμάτων Εναλλακτικών Απαντήσεων

Σε αυτή τη περίπτωση τώρα παρατηρούμε πως όσο μεγαλύτερο είναι το Temperature και το Top-p, τόσο μεγαλύτερη ποικιλία και διαφορά βλέπουμε στις απαντήσεις, που χωρίς αυτές τις 2 παραμέτρους το μοντέλο απαντάει σχεδόν ντετερμινιστικά.

To Temperature συμβάλει στο πόσο ντετερμινιστικά θα απαντήσει το μοντέλο, ενώ το Top-p θέτει το όριο της πιθανότητας των πιο πιθανών λέξεων από το οποίο θα επιλέξει ένα μοντέλο για την απάντηση του.

## Ομοιώτητα Ενσωματώσεων

In [ ]:
def embedding_similarity(llm: Llama):
    sentences = [
        "Artificial Intelligence is transforming the job market.",
        "A car crash can be a traumatic experience.",
        "Alchohol is bad for your health.",
        "Deep learning is used in image recognition.",
        "Greece is known for it's delicious food.",
    ]

    embeds = [llm.create_embedding(input=s)["data"][0] for s in sentences]
    embeddings = [np.array(embed["embedding"], dtype=np.float32).flatten() for embed in embeds]
    
    # Pad embeddings with mean to the same length
    max_len = max([len(embed) for embed in embeddings])
    embeddings = [np.pad(embed, (0, max_len - len(embed)), mode='mean') for embed in embeddings]

    # print([len(embed) for embed in embeddings])

    # Compute cosine similarity matrix
    similarity_matrix = cosine_similarity(np.array(embeddings))

    print("\n--- Cosine Similarity Matrix ---")
    size = len(sentences)
    df = pd.DataFrame(similarity_matrix, 
                      columns=[f"Sentence {i+1}" for i in range(size)], 
                      index=[f"Sentence {i+1}" for i in range(size)])
    print(df.to_string())

embedding_similarity(get_llm("Aya", embed=True))

llama_new_context_with_model: n_ctx_per_seq (256) < n_ctx_train (8192) -- the full capacity of the model will not be utilized



--- Cosine Similarity Matrix ---
            Sentence 1  Sentence 2  Sentence 3  Sentence 4  Sentence 5
Sentence 1    1.000001    0.231920    0.190776    0.515325    0.255183
Sentence 2    0.231920    1.000001    0.270016    0.234465    0.279600
Sentence 3    0.190776    0.270016    1.000001    0.201567    0.252004
Sentence 4    0.515325    0.234465    0.201567    1.000000    0.230058
Sentence 5    0.255183    0.279600    0.252004    0.230058    1.000002


### Σχολιασμός Ομοιότητας Προτάσεων

Παρατηρούμε πως η ομοιότητα συνημιτόνου πάνω στις ενσωματώσεις των προτάσεων κάνει μια σχετικά καλή δουλεία για την αναγνώριση όμοιων εννοιών μέσα στις προτάσεις. Αυτό είναι ένα πολύ χρήσιμο χαρακτηριστικό για την ανάλυση της σημασιολογίας των προτάσεων.

Έτσι από τα αποτελέσματα μπορούμε να δούμε πως το μοντέλο Aya αναγνωρίζει πως υπάρχει μεγάλη ομοιώτητα μεταξύ των εννοιών μεταξύ της Τεχνητής Νοημοσύνης και της Βαθιάς Μάθησης ή πως το αλκοόλ είναι ένας παράγοντας που επηρεάζει την οδήγηση. Περιέργως όμως πολλές προτάσεις που δοκίμασα είχαν γενικότερα υψηλότερη ομοιότητα με την 1η πρόταση, όπως για παράδειγμα με την 5η πρόταση που έχει μεγαλύτερη ομοιότητα με την 1η πρόταση από ότι με την 3η πρόταση όπως θα ήταν πιο λογικό.

Επίσης, το μοντέλο Aya φαίνεται να είναι πιο αποδοτικό στην ανάλυση της σημασιολογίας των προτάσεων σε σχέση με το μοντέλο Llama. Αυτό πιθανός να οφείλεται στο γεγονός ότι το μοντέλο Aya είναι πιο βαθή από το Llama.